In [1]:
import numpy as np
import torch
PATH_annotations = "C:\\Users\\t1526\\Desktop\\kaggle\\input\\benetech-making-graphs-accessible\\train\\annotations\\"
PATH_images = "C:\\Users\\t1526\\Desktop\\kaggle\\input\\benetech-making-graphs-accessible\\train\\images\\"
CHART_TYPE = {'vertical_bar':0,'horizontal_bar':1,'dot':2,'line':3,'scatter':4}
OBJ_TYPE = {'label_angle00':5,'label_angle45':6,'label_angle90':7,'label_angle-45':8,
            'axis_x':9,'axis_y':10,'tick_point_x':11,'tick_point_y':12,'useless':13}
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import json, os
_,_,annotation_list = next(os.walk(PATH_annotations))
_,_,image_list = next(os.walk(PATH_images))
annotation_list.sort()
image_list.sort()
annotation_list_train = annotation_list[:57500]
annotation_list_test = annotation_list[57500:]
image_list_train = image_list[:57500]
image_list_test = image_list[57500:]

In [6]:
from PIL import Image
from shutil import copy

for idx in annotation_list:
    
    try:
    
        string = ""
        # read annotation message
        with open(PATH_annotations+idx) as file:
            this_annotation = json.loads(file.read())
        if this_annotation['chart-type']!='scatter': continue
        # get image message
        img = Image.open(PATH_images+idx.split(".")[0]+".jpg")
        image_width, image_height = img.size
        plot_bb = this_annotation['plot-bb']
        chart_x0 = plot_bb['x0']
        chart_y0 = plot_bb['y0']
        chart_x1 = plot_bb['x0']+plot_bb['width']
        chart_y1 = plot_bb['y0']+plot_bb['height']
        # main_chart = img.crop(box=(chart_x0,chart_y0,chart_x1,chart_y1))

        # x 
        point1 = {'u':None,'v':None,'x':None,'y':None}
        point2 = {'u':None,'v':None,'x':None,'y':None}
        point3 = {'u':None,'v':None,'x':None,'y':None}
        point4 = {'u':None,'v':None,'x':None,'y':None}

        x_max= -1000000000000000; x_min= 1000000000000000
        x_max_id = -1; x_min_id=-1
        for i in this_annotation['axes']['x-axis']['ticks']:
            if i['tick_pt']['x']>x_max:
                x_max = i['tick_pt']['x']
                x_max_id = i['id']
        for i in this_annotation['axes']['x-axis']['ticks']:
            if i['tick_pt']['x']<x_min:
                x_min = i['tick_pt']['x']
                x_min_id = i['id']
        y_max= -1000000000000000; y_min= 1000000000000000
        y_max_id = -1; y_min_id=-1      
        for i in this_annotation['axes']['y-axis']['ticks']:
            if i['tick_pt']['y']>y_max:
                y_max = i['tick_pt']['y']
                y_max_id = i['id']
        for i in this_annotation['axes']['y-axis']['ticks']:
            if i['tick_pt']['y']<y_min:
                y_min = i['tick_pt']['y']
                y_min_id = i['id']
        point1['y']=y_min
        point2['y']=y_max
        point3['x']=x_min
        point4['x']=x_max
        for i in this_annotation['text']:
            if i['id']==x_max_id:
                point4['u']=float(i['text'])
                break
        for i in this_annotation['text']:
            if i['id']==x_min_id:
                point3['u']=float(i['text'])
                break
        for i in this_annotation['text']:
            if i['id']==y_max_id:
                point2['v']=float(i['text'])
                break
        for i in this_annotation['text']:
            if i['id']==y_min_id:
                point1['v']=float(i['text'])
                break
        # img.show()

        string = ""
        for point in this_annotation['data-series']:
            u0 = point['x']
            v0 = point['y']

            u2x = lambda u:(point3['x']-point4['x'])/(point3['u']-point4['u'])*u+\
                (point3['u']*point4['x']-point4['u']*point3['x'])/(point3['u']-point4['u'])
            v2y = lambda v:(point1['y']-point2['y'])/(point1['v']-point2['v'])*v+\
                (point1['v']*point2['y']-point2['v']*point1['y'])/(point1['v']-point2['v'])

            x0 = u2x(u0)/image_width
            y0 = v2y(v0)/image_height
            width = 0.030
            height = 0.030

            string += f"0 {x0} {y0} {width} {height}\n"

        with open(f"./labels/{idx.split('.')[0]}.txt",'w') as file:
            file.write(string)
        copy(PATH_images+idx.split(".")[0]+".jpg", "./images/"+idx.split(".")[0]+".jpg")
        # img = img.crop((u2x(u0)-5,v2y(v0)-5,u2x(u0)+5,v2y(v0)+5))
        # img.save("./x.jpg")
        #print(x0,y0,u2x(u0),v2y(v0))
    except:
        print(idx)


0278b786a87e.json
1a612638c6c7.json
1ab7f626447d.json
1f0ce7d70104.json
241b5d921b2f.json
2aa9d94f370a.json
312313d42699.json
3558120bff86.json
3c3506343922.json
43f6a39f9750.json
487b616d01b5.json
6264bd755db0.json
648809c3743b.json
88bf97c50204.json
90c504a8b320.json
90eaf488a44d.json
944014b8ce8c.json
9b62dbdded7b.json
9c924e860cc0.json
b99a20b437b4.json
c1bff8de06a9.json
cae276495445.json
d05c03df37eb.json
d75a8f2a99db.json
d767b5908804.json
e7fef7a144e7.json
ebc368b17b7a.json


In [7]:
_,_,image_list = next(os.walk("./images/"))
image_list_train = image_list[:10000]
image_list_test = image_list[10000:]
string = ""
for i in image_list_train:
    string+= "C:\\Users\\t1526\\Desktop\\kaggle\\working\\train\\scatter_pieces\\images\\"+i+"\n"
with open(f"./train.txt",'w') as file:
    file.write(string)


In [8]:
_,_,image_list = next(os.walk("./images/"))
image_list_train = image_list[:10000]
image_list_test = image_list[10000:]
string = ""
for i in image_list_test:
    string+= "C:\\Users\\t1526\\Desktop\\kaggle\\working\\train\\scatter_pieces\\images\\"+i+"\n"
with open(f"./test.txt",'w') as file:
    file.write(string)


In [9]:
_,_,image_list = next(os.walk("./images/"))
image_list_train = image_list[:10000]
image_list_test = image_list[10000:]
string = ""
for i in image_list_test:
    string+= "C:\\Users\\t1526\\Desktop\\kaggle\\working\\train\\scatter_pieces\\images\\"+i+"\n"
with open(f"./eval.txt",'w') as file:
    file.write(string)